In [1]:
import numpy as np

In [2]:
import pandas as pd
data_path = './'
data_id = pd.read_csv(data_path + 'data_identification.csv')
emotions = pd.read_csv(data_path + 'emotion.csv')
sample_sub = pd.read_csv(data_path + 'sampleSubmission.csv')
tweets = pd.read_json(data_path + 'tweets_DM.json', lines=True)
tweets_important = pd.DataFrame(tweets._source)

In [3]:
# Create a final dataframe from all of the data sources
tw_list = tweets_important['_source'].to_list()
tmp_df = pd.DataFrame.from_records(tw_list)
tmp_df_list = tmp_df['tweet'].to_list()
final_tweet_df = pd.DataFrame.from_records(tmp_df_list)
df_final = pd.merge(final_tweet_df, data_id, how='outer', on='tweet_id').merge(emotions, how='outer', on='tweet_id')

df_final.head()

,hashtags,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,NaN
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,NaN


In [4]:
df_final.shape

(1867535, 5)

In [6]:

# for 
emotion2idx = {}
idx2emotion = {}
for i, emotion in enumerate(df_final['emotion'].unique()):
    emotion2idx[emotion] = i
    idx2emotion[i] = emotion
    
print(emotion2idx)
print(idx2emotion)

{'anticipation': 0, 'sadness': 1, nan: 2, 'fear': 3, 'joy': 4, 'anger': 5, 'trust': 6, 'disgust': 7, 'surprise': 8}
{0: 'anticipation', 1: 'sadness', 2: nan, 3: 'fear', 4: 'joy', 5: 'anger', 6: 'trust', 7: 'disgust', 8: 'surprise'}


In [7]:
# Preprocess the tweet (removes hashtag and references (@))
import preprocessor as p # needs tweet-preprocessor package
# def preprocess_tweet(text):
#     text = p.clean(text)
#     text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('<lh>', '').strip()
#     return text


def preprocess_tweet(text):
    new_text = []
    for t in text.split():
        if t == '<LH>':
            continue
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        t = t.lower()
        
        
        new_text.append(t)
    return " ".join(new_text)



In [8]:


# for i in range(100):
#     d = df_final.iloc[i]
#     # print(d)
#     t = d.text
# #     print(t)
#     print(preprocess_tweet(t))
    

In [9]:
df_final['text'] = df_final['text'].apply(preprocess_tweet)

In [10]:
train_df = df_final[df_final['identification'] == 'train']
test_df = df_final[df_final['identification'] == 'test']

In [11]:
train_df.head()

,hashtags,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"people who post ""add me on #snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@user as we see, trump is dangerous to #freepr...",train,sadness
3,[],0x1cd5b0,now issa is stalking tasha 😂😂😂,train,fear
5,"[authentic, LaughOutLoud]",0x1d755c,@user @user thx for the best time tonight. wha...,train,joy
6,[],0x2c91a8,still waiting on those supplies liscus.,train,anticipation


In [12]:
print(train_df.shape[0])
print(test_df.shape[0])

1455563
411972


In [13]:
# train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv', index=False)

In [78]:
df_final.emotion.value_counts()

joy             516017
anticipation    248935
trust           205478
sadness         193437
disgust         139101
fear             63999
surprise         48729
anger            39867
Name: emotion, dtype: int64

In [77]:
for x in df_final.emotion.value_counts():
    print(i,x)

TypeError: cannot unpack non-iterable int object

In [68]:
train_df = train_df.replace('NaN', np.nan)
train_df = train_df.dropna(subset=['emotion'])
print(len(train_df))

1455563


In [71]:
train_df = pd.read_csv('train.csv')
train_df

,hashtags,tweet_id,text,identification,emotion
0,['Snapchat'],0x376b20,"people who post ""add me on #snapchat"" must be ...",train,anticipation
1,"['freepress', 'TrumpLegacy', 'CNN']",0x2d5350,"@user as we see, trump is dangerous to #freepr...",train,sadness
2,[],0x1cd5b0,now issa is stalking tasha 😂😂😂,train,fear
3,"['authentic', 'LaughOutLoud']",0x1d755c,@user @user thx for the best time tonight. wha...,train,joy
4,[],0x2c91a8,still waiting on those supplies liscus.,train,anticipation
...,...,...,...,...,...
1455558,"['NoWonder', 'Happy']",0x321566,i'm so happy!!! #nowonder the name of this sho...,train,joy
1455559,[],0x38959e,in every circumtance i'd like to be thankful t...,train,joy
1455560,['blessyou'],0x2cbca6,there's currently two girls walking around the...,train,joy
1455561,[],0x24faed,"ah, corporate life, where you can date using j...",train,joy
